In [ ]:
dbutils.fs.ls("/FileStore/tables")

Out[30]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000),
 FileInfo(path='dbfs:/FileStore/tables/geo_dirty.csv/', name='geo_dirty.csv/', size=0, modificationTime=1701794298958),
 FileInfo(path='dbfs:/FileStore/tables/pin_dirty.csv/', name='pin_dirty.csv/', size=0, modificationTime=1701794298958),
 FileInfo(path='dbfs:/FileStore/tables/user_dirty.csv/', name='user_dirty.csv/', size=0, modificationTime=1701794298958)]

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("dbfs:/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0abb070c336b-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0abb070c336b-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
#dbutils.fs.unmount("/mnt/mount_s3_to_db")

Out[88]: True

In [ ]:
display(dbutils.fs.ls("/mnt/user-0abb070c336b-bucket/topics"))


dbfs:/mnt/user-0abb070c336b-bucket/topics/0abb070c336b.geo/ | 0abb070c336b.geo/ | 0 | 1701799366317
dbfs:/mnt/user-0abb070c336b-bucket/topics/0abb070c336b.pin/ | 0abb070c336b.pin/ | 0 | 1701799366317
dbfs:/mnt/user-0abb070c336b-bucket/topics/0abb070c336b.user/ | 0abb070c336b.user/ | 0 | 1701799366317

In [ ]:
# List of topics
list_of_topics = [".pin", ".geo", ".user"]

def read_topic_to_df(topic):
    # path to files in topic
    file_location = f"/mnt/user-0abb070c336b-bucket/topics/0abb070c336b{topic}/partition=0/*.json"
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    return df

# Create DataFrames for each topic
for item in list_of_topics:
    # Name the DataFrames using statement strings
    df_name = f"df_{item[1:]}"
    # Execute the function and assign the result to the variable
    exec(f"{df_name} = read_topic_to_df('{item}')")

# Display the created DataFrames
for item in list_of_topics:
    # Show the DataFrames using statement strings
    show_df_statement = f"display(df_{item[1:]})"
    exec(show_df_statement)

df_pin:pyspark.sql.dataframe.DataFrame = [age: long, category: string ... 13 more fields]
df_geo:pyspark.sql.dataframe.DataFrame = [country: string, ind: long ... 3 more fields]
df_user:pyspark.sql.dataframe.DataFrame = [age: long, date_joined: string ... 3 more fields]